In [ ]:
!pip install yt-dlp mediapipe opencv-python
!mkdir frames

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.0 MB/s eta 0:0

In [ ]:
import yt_dlp

ydl_opts = {
    'format': 'best',
    'outtmpl': 'video.mp4'
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(["https://www.youtube.com/watch?v=Jlp8G9paliw"])  # ← replace with your video link


[youtube] Extracting URL: https://www.youtube.com/watch?v=Jlp8G9paliw
[youtube] Jlp8G9paliw: Downloading webpage


[youtube] Jlp8G9paliw: Downloading android sdkless player API JSON
[youtube] Jlp8G9paliw: Downloading web safari player API JSON


[youtube] Jlp8G9paliw: Downloading m3u8 information


[info] Jlp8G9paliw: Downloading 1 format(s): 96
[download] Sleeping 6.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 8
[download] Destination: video.mp4
[download] 100% of    7.05MiB in 00:00:00 at 18.12MiB/s                
[FixupM3u8] Fixing MPEG-TS in MP4 container of "video.mp4"


In [12]:
import cv2
import mediapipe as mp
import math
import os
import shutil

# ---------------- Helper Functions ---------------- #
def distance(p1, p2):
    return math.dist(p1, p2)

# ---------------- Extract Frames ---------------- #
cap = cv2.VideoCapture("video.mp4")
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
sample_frames = sorted([int(total_frames*i/6) for i in range(1,6)])  # 5 representative frames

frame_idx = 0
saved_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx in sample_frames:
        path = f"representative/frame_{frame_idx}.jpg"
        cv2.imwrite(path, frame)
        saved_frames.append((frame_idx, frame))
    frame_idx += 1
cap.release()

print("Representative frames saved:", [f[0] for f in saved_frames])

# ---------------- Run MediaPipe Pose ---------------- #
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose_model = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

pose_keypoints = {}

for idx, frame in saved_frames:
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose_model.process(frame_rgb)
    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imwrite(f"pose_representative/frame_{idx}.jpg", frame)
        # Save nose keypoint (ID=0) and all points
        pose_keypoints[idx] = {id:(lm.x*frame.shape[1], lm.y*frame.shape[0]) for id,lm in enumerate(result.pose_landmarks.landmark)}

pose_model.close()
print("Pose-overlaid frames saved.")

# ---------------- Compute Head Movement Distance ---------------- #
metric_name = "Head Movement Distance (nose keypoint)"
head_movements = []

sorted_frames = sorted(pose_keypoints.keys())
for i in range(1, len(sorted_frames)):
    f1 = sorted_frames[i-1]
    f2 = sorted_frames[i]
    nose1 = pose_keypoints[f1][0]
    nose2 = pose_keypoints[f2][0]
    dist_val = distance(nose1, nose2)
    head_movements.append((f1, f2, dist_val))

# ---------------- Write Descriptions ---------------- #
descriptions = []
for f in sorted_frames:
    desc = f"Frame {f}: visually "
    # Rough approximation using frame index to describe pose (replace with your observation if manual)
    if f==sorted_frames[0]:
        desc += "initial pose."
    elif f==sorted_frames[-1]:
        desc += "final pose."
    else:
        desc += "middle motion pose."
    descriptions.append(desc)

print("\n--- Frame Descriptions ---")
for d in descriptions:
    print(d)

print("\n--- Head Movement Distances ---")
for f1, f2, dist_val in head_movements:
    print(f"Between Frame {f1} and Frame {f2}: {dist_val:.1f} pixels")

print("\nMetric used:", metric_name)
print("Frames used:", sorted_frames)


Representative frames saved: [157, 315, 472, 630, 787]
Pose-overlaid frames saved.

--- Frame Descriptions ---
Frame 157: visually initial pose.
Frame 315: visually middle motion pose.
Frame 472: visually middle motion pose.
Frame 630: visually final pose.

--- Head Movement Distances ---
Between Frame 157 and Frame 315: 20.2 pixels
Between Frame 315 and Frame 472: 23.3 pixels
Between Frame 472 and Frame 630: 150.0 pixels

Metric used: Head Movement Distance (nose keypoint)
Frames used: [157, 315, 472, 630]


In [14]:
# Install git if not installed
!apt-get install git -qq

# Configure your Git username/email (replace with your info)
!git config --global user.name "Menaria123"
!git config --global user.email "menariakusum8@gmail.com"


In [15]:
!git clone https://github.com/Menaria123/Motion-Detection-.git

Cloning into 'Motion-Detection-'...


In [18]:
!cp -r representative pose-analysis/
!cp -r pose_representative pose-analysis/
!git add .
!git commit -m "Add representative and pose frames"
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [22]:
import shutil



# Copy representative frames
shutil.copytree("representative", "Motion-Detection-/representative", dirs_exist_ok=True)

# Copy pose-overlaid frames
shutil.copytree("pose_representative", "Motion-Detection-/pose_representative", dirs_exist_ok=True)


'Motion-Detection-/pose_representative'

In [23]:
%cd Motion-Detection-

# Stage all files
!git add .

# Commit
!git commit -m "Add Colab pose estimation pipeline, representative frames, pose-overlays"

# Push to GitHub (replace with Personal Access Token if needed)
!git push origin main

/content/Motion-Detection-
[main (root-commit) a50ef90] Add Colab pose estimation pipeline, representative frames, pose-overlays
 9 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 pose_representative/frame_157.jpg
 create mode 100644 pose_representative/frame_315.jpg
 create mode 100644 pose_representative/frame_472.jpg
 create mode 100644 pose_representative/frame_630.jpg
 create mode 100644 representative/frame_157.jpg
 create mode 100644 representative/frame_315.jpg
 create mode 100644 representative/frame_472.jpg
 create mode 100644 representative/frame_630.jpg
 create mode 100644 representative/frame_787.jpg
fatal: could not read Username for 'https://github.com': No such device or address
